In [1]:
import pandas as pd

Chargement et prétraitement des données

In [46]:
# Chemin vers le fichier CSV
file_path = "/Users/melusinecaillard/RSPL/data/cleaned_merged_seasons.csv"

In [52]:
import pandas as pd

def load_and_preprocess_data(file_path):
    # Chargement du jeu de données
    original_data = pd.read_csv(file_path, low_memory=False)

    # Créer une copie exacte du DataFrame
    data = original_data.copy()

    # Suppression des doublons et des valeurs manquantes
    data.drop_duplicates(inplace=True)
    data.dropna(inplace=True)

    return data


In [53]:
print(data.head)

<bound method NDFrame.head of                            name position         team_x  goals_scored  \
19852            Aaron Connolly      FWD       Brighton             0   
19853           Aaron Cresswell      DEF       West Ham             0   
19854                Aaron Mooy      MID       Brighton             0   
19855            Aaron Ramsdale       GK  Sheffield Utd             0   
19856        Abdoulaye Doucouré      MID        Everton             0   
...                         ...      ...            ...           ...   
96164              Oliver Skipp      MID          Spurs             0   
96165            Ryan Sessegnon      DEF          Spurs             0   
96166              Ashley Young      DEF    Aston Villa             0   
96167  Jeremy Sarmiento Morante      MID       Brighton             0   
96168            Philip Billing      MID    Bournemouth             0   

       assists  clean_sheets  yellow_cards  red_cards  
19852        0             0         

Ingénierie des caractéristiques

In [63]:
def feature_engineering(data):
    # Sélection des caractéristiques pertinentes
    selected_features = ['name', 'position', 'team_x', 'goals_scored', 'assists', 'clean_sheets', 'yellow_cards', 'red_cards']
    
    # Filtrage des caractéristiques sélectionnées
    data = data[selected_features].copy()  # Crée une copie explicite du DataFrame

    # Ajout de nouvelles caractéristiques si nécessaire (par exemple, la forme récente du joueur)
    # Ici, nous allons ajouter une caractéristique pour le total des cartons
    data['total_cards'] = data['yellow_cards'] + data['red_cards']

    return data

# Chemin vers le fichier CSV
file_path = "/Users/melusinecaillard/RSPL/data/cleaned_merged_seasons.csv"

# Chargement et prétraitement des données
data = load_and_preprocess_data(file_path)

# Ingénierie des caractéristiques
data = feature_engineering(data)

In [64]:
# Affichage des premières lignes du jeu de données après prétraitement et ingénierie des caractéristiques
print(data.head())

                     name position         team_x  goals_scored  assists  \
19852      Aaron Connolly      FWD       Brighton             0        0   
19853     Aaron Cresswell      DEF       West Ham             0        0   
19854          Aaron Mooy      MID       Brighton             0        0   
19855      Aaron Ramsdale       GK  Sheffield Utd             0        0   
19856  Abdoulaye Doucouré      MID        Everton             0        0   

       clean_sheets  yellow_cards  red_cards  total_cards  
19852             0             0          0            0  
19853             0             0          0            0  
19854             0             0          0            0  
19855             0             0          0            0  
19856             1             0          0            0  


Modélisation et contraintes budget et compo

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [71]:
# Fonction pour diviser les données en ensembles d'entraînement et de test
def split_data(data):
    X = data.drop(columns=['goals_scored', 'assists'])  # Caractéristiques sans les performances
    y = data[['goals_scored', 'assists']]  # Performances à prédire

    
    # Encoder les variables catégorielles
    X_encoded = pd.get_dummies(X)

    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test


In [72]:
# Fonction pour entraîner un modèle de régression linéaire
def train_linear_regression_model(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model

In [73]:
# Fonction pour prédire les performances des joueurs avec contraintes de budget et de composition d'équipe
def predict_player_performance(model, player_features, budget):
    # Prédiction des performances du joueur
    predicted_performance = model.predict(player_features)

    # Vérifier si le joueur respecte les contraintes de budget
    total_cost = np.sum(player_features['cost'])
    if total_cost > budget:
        return "Le joueur sélectionné dépasse le budget."
    else:
        return predicted_performance

In [74]:
# Chargement du jeu de données prétraité et ingénierie des caractéristiques
data = load_and_preprocess_data("/Users/melusinecaillard/RSPL/data/cleaned_merged_seasons.csv")
data = feature_engineering(data)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = split_data(data)

In [75]:
# Entraînement du modèle de régression linéaire
linear_regression_model = train_linear_regression_model(X_train, y_train)

In [76]:
# Prédiction des performances des joueurs pour l'ensemble de test
y_pred = linear_regression_model.predict(X_test)

In [77]:
# Évaluation de la performance du modèle
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 826738119213.1672


In [78]:
# Exemple de prédiction pour un joueur avec un budget donné
player_features_example = pd.DataFrame([[...]], columns=X_train.columns)  # Remplacez ... par les caractéristiques du joueur
budget_example = 100  # Exemple de budget
predicted_performance_example = predict_player_performance(linear_regression_model, player_features_example, budget_example)
print("Prédiction de performances du joueur:", predicted_performance_example)

ValueError: 1361 columns passed, passed data had 1 columns

Recommandations players, captain & vice-capt

In [ ]:
# Fonction pour recommander les meilleurs joueurs disponibles en respectant les contraintes
def recommend_players(model, data, budget, team_constraints, captain=True):
    # Filtrer les joueurs disponibles en fonction du budget et des contraintes d'équipe
    available_players = data[(data['cost'] <= budget) & (data.groupby('team')['player_name'].transform('count') <= team_constraints)]

    # Prédiction des performances des joueurs disponibles
    predicted_performances = model.predict(available_players.drop(columns=['goals_scored', 'assists']))

    # Sélection des meilleurs joueurs en fonction des performances prédites
    available_players['predicted_goals'] = predicted_performances[:, 0]
    available_players['predicted_assists'] = predicted_performances[:, 1]
    available_players['total_predicted_points'] = 2 * available_players['predicted_goals'] + available_players['predicted_assists']

    # Trier les joueurs par performances prédites
    sorted_players = available_players.sort_values(by='total_predicted_points', ascending=False)

    # Sélectionner le capitaine et le vice-capitaine
    captain_player = sorted_players.iloc[0]  # Capitaine est le meilleur joueur
    vice_captain_player = sorted_players.iloc[1]  # Vice-capitaine est le deuxième meilleur joueur

    # Retourner les recommandations de joueurs et les informations sur le capitaine et le vice-capitaine
    return sorted_players, captain_player, vice_captain_player

In [ ]:
# Budget et contraintes d'équipe
budget = 100  # Exemple de budget
team_constraints = 3  # Maximum de joueurs de la même équipe

# Recommander les joueurs en respectant les contraintes
recommended_players, captain_player, vice_captain_player = recommend_players(linear_regression_model, data, budget, team_constraints)

In [ ]:
# Afficher les recommandations de joueurs
print("Recommandations de joueurs:")
print(recommended_players.head())

In [ ]:
# Afficher les informations sur le capitaine et le vice-capitaine
print("\nCapitaine recommandé:")
print(captain_player)
print("\nVice-capitaine recommandé:")
print(vice_captain_player)

Gestion des transferts

In [ ]:
# Fonction pour effectuer un transfert tout en respectant les contraintes
def make_transfer(current_team, new_player, budget, team_constraints):
    # Vérifier si le joueur transféré respecte les contraintes de budget et d'équipe
    if new_player['cost'] <= budget and current_team.groupby('team')['player_name'].transform('count').max() < team_constraints:
        # Ajouter le nouveau joueur à l'équipe
        updated_team = current_team.append(new_player)
        # Mettre à jour le budget restant
        updated_budget = budget - new_player['cost']
        return updated_team, updated_budget
    else:
        return current_team, budget  # Aucun changement dans l'équipe et le budget

In [ ]:
# Exemple de fonction pour effectuer des transferts hebdomadaires
def weekly_transfers(current_team, budget, team_constraints, transfer_player, new_player):
    # Effectuer le transfert
    updated_team, updated_budget = make_transfer(current_team, new_player, budget, team_constraints)
    
    # Mettre à jour les recommandations en fonction des transferts effectués
    recommended_players, _, _ = recommend_players(linear_regression_model, data, updated_budget, team_constraints)

    return updated_team, updated_budget, recommended_players

In [ ]:
# Exemple de jeu d'équipe actuel
current_team = pd.DataFrame([["Player1", "Team1", 10]], columns=['player_name', 'team', 'cost'])
# Exemple de joueur à transférer
transfer_player = {"player_name": "Player1", "team": "Team1", "cost": 10}
# Exemple de nouveau joueur à ajouter
new_player = {"player_name": "Player2", "team": "Team2", "cost": 8}

In [ ]:
# Exemple d'exécution des transferts hebdomadaires
updated_team, updated_budget, updated_recommendations = weekly_transfers(current_team, budget, team_constraints, transfer_player, new_player)

In [ ]:
# Afficher l'équipe mise à jour
print("Équipe mise à jour après les transferts:")
print(updated_team)

In [ ]:
# Afficher le budget restant après les transferts
print("\nBudget restant après les transferts:", updated_budget)

In [ ]:
# Afficher les recommandations de joueurs mises à jour après les transferts
print("\nRecommandations de joueurs mises à jour:")
print(updated_recommendations.head())

Évaluation et ajustement

In [ ]:
# Fonction pour évaluer l'algorithme en utilisant des métriques
def evaluate_algorithm(predictions, actual):
    # Calculer les métriques d'évaluation (par exemple, précision, rappel, score F1)
    # Ici, nous calculons simplement la précision pour illustrer l'exemple
    correct_predictions = predictions == actual
    accuracy = correct_predictions.mean()
    return accuracy

# Générer des prédictions pour l'ensemble de test
predictions = model.predict(X_test)

In [ ]:
# Évaluer l'algorithme
accuracy = evaluate_algorithm(predictions, y_test)
print("Précision de l'algorithme:", accuracy)